In [ ]:
import pandas as pd

from custom_dataset import ImageLoaderCustom
from parameters import DEVICE


In [ ]:
# csv containing Q_table
Q_Table_nonfoveated = pd.read_csv('Q_tables/Q_table_nonfoveated.csv', sep=',')
Q_Table_strong = pd.read_csv('Q_tables/Q_table_strongfoveated.csv', sep=',')
Q_Table_weak = pd.read_csv('Q_tables/Q_table_weakfoveated.csv', sep=',')

